In [52]:
import pandas as pd
import numpy as np

In [53]:
# request user to input model name
model_name = input("Please enter the model name of type \"model_<identifier>\": ")

In [54]:
print(f"Model name set to: {model_name}")

Model name set to: model_02r7


In [55]:
val_results_folder = "/users/1/lundq163/projects/automated-qc/doc/models/{}/".format(model_name)
print(f"Validation results folder set to: {val_results_folder}")

Validation results folder set to: /users/1/lundq163/projects/automated-qc/doc/models/model_02r7/


In [56]:
# for available fold csvs, merge them into one dataframe
all_folds_df = None
for fold in range(5):
    fold_csv_path = val_results_folder + "{}_fold_{}.csv".format(model_name, fold)
    fold_df = pd.read_csv(fold_csv_path)
    if all_folds_df is None:
        all_folds_df = fold_df.copy()
    else:
        all_folds_df = pd.merge(all_folds_df, fold_df, on='scan', how='outer', suffixes=('', f'_fold{fold}'))
        all_folds_df['predicted_qu_motion_score'] = all_folds_df['predicted_qu_motion_score'].fillna(all_folds_df[f'predicted_qu_motion_score_fold{fold}'])
        all_folds_df = all_folds_df.drop(columns=[f'predicted_qu_motion_score_fold{fold}', f'subject_id_fold{fold}', f'session_id_fold{fold}', f'run_id_fold{fold}', f'suffix_fold{fold}', f'QU_motion_fold{fold}', f'training_fold{fold}'])
all_folds_df.drop(columns=['training'], inplace=True)
all_folds_df.rename(columns={'validation': 'validation_fold0'}, inplace=True)
all_folds_df = all_folds_df[['subject_id', 'session_id', 'run_id', 'suffix', 'scan', 'QU_motion', 'predicted_qu_motion_score', 'validation_fold0', 'validation_fold1', 'validation_fold2', 'validation_fold3', 'validation_fold4']]

In [57]:
all_folds_df.dtypes

subject_id                    object
session_id                    object
run_id                         int64
suffix                        object
scan                          object
QU_motion                    float64
predicted_qu_motion_score    float64
validation_fold0               int64
validation_fold1               int64
validation_fold2               int64
validation_fold3               int64
validation_fold4               int64
dtype: object

In [58]:
all_folds_df

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,validation_fold0,validation_fold1,validation_fold2,validation_fold3,validation_fold4
0,sub-100079,ses-V02,1,T2w,sub-100079_ses-V02_run-1_T2w.nii.gz,3.0,1.989765,0,0,0,1,0
1,sub-100247,ses-V02,1,T2w,sub-100247_ses-V02_run-1_T2w.nii.gz,3.0,2.885908,0,1,0,0,0
2,sub-100524,ses-V02,1,T2w,sub-100524_ses-V02_run-1_T2w.nii.gz,1.0,0.794043,0,0,0,0,1
3,sub-100586,ses-V02,1,T2w,sub-100586_ses-V02_run-1_T2w.nii.gz,0.5,0.264855,0,0,0,0,1
4,sub-100895,ses-V02,1,T2w,sub-100895_ses-V02_run-1_T2w.nii.gz,1.0,0.907853,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1019,sub-206064,ses-V02,1,T2w,sub-206064_ses-V02_run-1_T2w.nii.gz,1.5,1.243339,1,0,0,0,0
1020,sub-206064,ses-V03,1,T2w,sub-206064_ses-V03_run-1_T2w.nii.gz,1.5,0.957190,1,0,0,0,0
1021,sub-206064,ses-V03,2,T2w,sub-206064_ses-V03_run-2_T2w.nii.gz,1.5,0.851769,1,0,0,0,0
1022,sub-206064,ses-V04,1,T1w,sub-206064_ses-V04_run-1_T1w.nii.gz,0.5,1.143729,1,0,0,0,0


In [59]:
#val_results = pd.read_csv('../../doc/models/model_02/model_02_fixed_scores.csv')
#val_results = pd.read_csv('../../doc/models/model_02r/model_02r6.csv')
#val_results = pd.read_csv('../../doc/models/model_02s/model_02s.csv')
#val_results = pd.read_csv('../../doc/models/model_07/model_07.csv')
#val_results = pd.read_csv('../../doc/models/model_02r/model_02r3_part2.csv')
#val_results.head()

In [60]:
val_results = all_folds_df

In [61]:
# find scans with highest prediction error
val_results['abs_error'] = abs(val_results['QU_motion'] - val_results['predicted_qu_motion_score'])
val_results = val_results.sort_values(by='abs_error', ascending=False)
val_results.head(25)

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,validation_fold0,validation_fold1,validation_fold2,validation_fold3,validation_fold4,abs_error
955,sub-201881,ses-V03,1,T2w,sub-201881_ses-V03_run-1_T2w.nii.gz,3.0,1.274987,0,0,1,0,0,1.725013
258,sub-130428,ses-V04,1,T2w,sub-130428_ses-V04_run-1_T2w.nii.gz,3.0,1.331271,0,0,0,0,1,1.668729
671,sub-170499,ses-V02,2,T2w,sub-170499_ses-V02_run-2_T2w.nii.gz,2.0,0.341013,0,0,0,1,0,1.658987
257,sub-130428,ses-V04,1,T1w,sub-130428_ses-V04_run-1_T1w.nii.gz,3.0,1.369446,0,0,0,0,1,1.630554
607,sub-161192,ses-V02,1,T2w,sub-161192_ses-V02_run-1_T2w.nii.gz,3.0,1.388291,0,1,0,0,0,1.611709
453,sub-146664,ses-V02,1,T2w,sub-146664_ses-V02_run-1_T2w.nii.gz,0.0,1.545455,0,0,1,0,0,1.545455
830,sub-185282,ses-V03,1,T2w,sub-185282_ses-V03_run-1_T2w.nii.gz,1.5,-0.042234,0,1,0,0,0,1.542234
817,sub-184065,ses-V04,1,T1w,sub-184065_ses-V04_run-1_T1w.nii.gz,2.0,0.497724,0,0,0,1,0,1.502276
615,sub-161454,ses-V02,2,T1w,sub-161454_ses-V02_run-2_T1w.nii.gz,1.0,2.469253,0,1,0,0,0,1.469253
362,sub-140409,ses-V03,1,T2w,sub-140409_ses-V03_run-1_T2w.nii.gz,2.0,0.535558,0,0,0,1,0,1.464442


In [62]:
# save first 25 rows to csv
val_results.head(25).to_csv('../../data/validation_errors/model_02r7/validation_set_top25_errors.csv', index=False)

In [63]:
# remove the 5 highest error scans
val_results = val_results.iloc[5:]
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,validation_fold0,validation_fold1,validation_fold2,validation_fold3,validation_fold4,abs_error
453,sub-146664,ses-V02,1,T2w,sub-146664_ses-V02_run-1_T2w.nii.gz,0.0,1.545455,0,0,1,0,0,1.545455
830,sub-185282,ses-V03,1,T2w,sub-185282_ses-V03_run-1_T2w.nii.gz,1.5,-0.042234,0,1,0,0,0,1.542234
817,sub-184065,ses-V04,1,T1w,sub-184065_ses-V04_run-1_T1w.nii.gz,2.0,0.497724,0,0,0,1,0,1.502276
615,sub-161454,ses-V02,2,T1w,sub-161454_ses-V02_run-2_T1w.nii.gz,1.0,2.469253,0,1,0,0,0,1.469253
362,sub-140409,ses-V03,1,T2w,sub-140409_ses-V03_run-1_T2w.nii.gz,2.0,0.535558,0,0,0,1,0,1.464442


In [64]:
# count NaN values in abs_error column
val_results['abs_error'].isna().sum()

np.int64(0)

In [65]:
# drop rows with NaN values in abs_error column
val_results = val_results.dropna(subset=['abs_error'])
val_results['abs_error'].isna().sum()

np.int64(0)

In [66]:
def create_correlation_coefficient(actual_vals, predicted_vals):
    """Calculate Pearson correlation coefficient between actual and predicted values.

    Args:
        actual_vals (list): Actual QU_motion scores
        predicted_vals (list): Predicted QU_motion scores

    Returns:
        float: Pearson correlation coefficient (r value)
    """
    x = np.array(actual_vals)
    y = np.array(predicted_vals)

    correlation_matrix = np.corrcoef(x, y)
    correlation_coefficient = correlation_matrix[0, 1]

    return correlation_coefficient

In [67]:
# recalculate metrics without the 5 highest error scans
corr = create_correlation_coefficient(val_results['QU_motion'], val_results['predicted_qu_motion_score'])
corr

np.float64(0.8188813921956978)